In [ ]:
#importing all the required libraries 
import speech_recognition as sr
from translate import Translator
import io, base64
from pydub import AudioSegment
from ttsmms import TTS 
import IPython.display as ipd

CASES OF INPUT FOR THE USER TO QUERY THE PDF:
1. HINDI SPEECH WILL BE USED
2. ENGLISH SPEECH WILL BE USED
3. HINDI TEXT WILL BE USED 
4. ENGLISH TEXT WILL BE USED

1. INPUT IN THE FORM OF HINDI SPEECH


In [ ]:
#reading the hindi audio file first and encoding it into base64 format, because audio file will be recieved in encoded base64 format from the react frontend
hindi_audio_path = ".\hindi_audio.mp3"
def encode_mp3_to_base64(file_path):
    with open(file_path, 'rb') as mp3_file:
        mp3_binary_data = mp3_file.read()
        base64_encoded = base64.b64encode(mp3_binary_data)
        return base64_encoded.decode('utf-8')

encoded_audio_hindi = encode_mp3_to_base64(hindi_audio_path)
encoded_audio_hindi

In [ ]:
#now decoding the audio file which was encoded above. THIS FUNCTIONALITY IS THE STARTING POINT IN THE BACKEND.
decoded_audio_hindi = base64.b64decode(encoded_audio_hindi)
decoded_audio_hindi

In [ ]:
#converting this decoded base64 into wav format.
audio_hindi = AudioSegment.from_mp3(io.BytesIO(decoded_audio_hindi))
wav_data_hindi = io.BytesIO()
audio_hindi.export(wav_data_hindi, format="wav")
wav_data_hindi.seek(0)
wav_converted_hindi = wav_data_hindi.read()

In [ ]:
#performing hindi speech to text conversion using python's speech_recognition library 
def wav_to_hindi_text(audio_content):
    recognizer = sr.Recognizer()
    audio_stream = io.BytesIO(audio_content)
    with sr.AudioFile(audio_stream) as source:
        audio = recognizer.record(source)
    text = recognizer.recognize_google(audio, language = 'hi-IN')
    return text

hindi_query_text = wav_to_hindi_text(wav_converted_hindi)
hindi_query_text

In [ ]:
#Performing hindi text to english translation as this text will be used for querying the pdf
#For this we will use python's translate library
def hindi_to_english_text(hindi_text):
    translator = Translator(to_lang="en", from_lang="hi")
    translated_text = translator.translate(hindi_text)
    return translated_text

english_query_text = hindi_to_english_text(hindi_query_text)
english_query_text

Now as in the flow, This english_query_text will be provided as a prompt for our LLM which will be trained on a pdf, the result provided by this LLM will now undergo conversions stated below.

In [ ]:
#As the LLM is not ready, let us assume the response from the LLM to be english_result_text
english_result_text = '''
An atom is the basic building block of matter, consisting of a nucleus composed of protons and neutrons, surrounded by electrons in orbit. The nucleus is at the center of the atom and contains positively charged protons and uncharged neutrons. Electrons, which are negatively charged, orbit the nucleus in shells or energy levels.
'''

In [ ]:
#Now we will translate this english_result_text in hindi, again using the translate library of python
def english_to_hindi_text(english_text):
    translator = Translator(to_lang="hi", from_lang='en')
    translated_text = translator.translate(english_text)
    return translated_text

hindi_result_text = english_to_hindi_text(english_result_text)

In [ ]:
#performing text-to-speech using python's ttsmms and a model which is stored in models folder.
def hindi_text_to_audio(text):
    dir_path = ("./models/hin")
    tts=TTS(dir_path)
    synthesized_wav=tts.synthesis(text)
    return synthesized_wav

hindi_result_audio = hindi_text_to_audio(hindi_result_text)
#displaying the audio:
ipd.Audio(hindi_result_audio["x"], rate=hindi_result_audio["sampling_rate"])

In [ ]:
#encoding the audio file to base64 format, this will be sent as an output to the frontend along the the hindi_result_text
def encoding_hindi_audio(synthesized_wav):
    wav=synthesized_wav["x"]
    wav_binary = wav.tobytes()
    base64_encoded = base64.b64encode(wav_binary).decode('utf-8')
    return base64_encoded
base64encodedaudio = encoding_hindi_audio(hindi_result_audio)
base64encodedaudio

2. INPUT IN THE FORM OF ENGLISH SPEECH

In [ ]:
#reading the hindi audio file first and encoding it into base64 format, because audio file will be recieved in encoded base64 format from the react frontend
english_audio_path = ".\english_audio.mp3"
encoded_audio_english = encode_mp3_to_base64(english_audio_path)
encoded_audio_english

In [ ]:
#now decoding the audio file which was encoded above. THIS FUNCTIONALITY IS THE STARTING POINT IN THE BACKEND.
decoded_audio_english = base64.b64decode(encoded_audio_english)
decoded_audio_english

In [ ]:
#converting this decoded base64 into wav format.
audio_english = AudioSegment.from_mp3(io.BytesIO(decoded_audio_english))
wav_data_english = io.BytesIO()
audio_english.export(wav_data_english, format="wav")
wav_data_english.seek(0)
wav_converted_english = wav_data_english.read()

In [ ]:
#performing english speech to text conversion using python's speech_recognition library 
def wav_to_english_text(audio_content):
    recognizer = sr.Recognizer()
    audio_stream = io.BytesIO(audio_content)
    with sr.AudioFile(audio_stream) as source:
        audio = recognizer.record(source)
    text = recognizer.recognize_google(audio, language = 'en-US')
    return text

english_query_text = wav_to_english_text(wav_converted_english)
english_query_text

Now as in the flow, This english_query_text will be provided as a prompt for our LLM which will be trained on a pdf, the result provided by this LLM will now undergo conversions stated below.

In [ ]:
#As the LLM is not ready, let us assume the response from the LLM to be english_result_text
english_result_text = '''
An atom is the basic building block of matter, consisting of a nucleus composed of protons and neutrons, surrounded by electrons in orbit. The nucleus is at the center of the atom and contains positively charged protons and uncharged neutrons. Electrons, which are negatively charged, orbit the nucleus in shells or energy levels.
'''

In [ ]:
#performing text-to-speech using python's ttsmms and a model which is stored in models folder.
def english_text_to_audio(text):
    dir_path = ("./models/eng")
    tts=TTS(dir_path)
    synthesized_wav=tts.synthesis(text)
    return synthesized_wav

english_result_audio = english_text_to_audio(english_result_text)
#displaying the audio:
ipd.Audio(english_result_audio["x"], rate=english_result_audio["sampling_rate"])

3. INPUT IN THE FORM OF HINDI TEXT

In [ ]:
#let's assume the input text entered by the user is hindi_query_text:
hindi_query_text = "नमस्ते, मैं आरिफ हूं और मैं वर्तमान में एक प्रोजेक्ट पर काम कर रहा हूं जो लैंगचेन, क्रोमाडीबी और हगिंगफेस का उपयोग करता है"

In [ ]:
english_query_text = hindi_to_english_text(hindi_query_text)
english_query_text 

Now as in the flow, This english_text will be provided as a prompt for our LLM which will be trained on a pdf, the result provided by this LLM will now undergo conversions stated below.

In [ ]:
#As the LLM is not ready, let us assume the response from the LLM to be english_result_text, this result will then be given to the user as the final result displayed.
english_result_text = '''
An atom is the basic building block of matter, consisting of a nucleus composed of protons and neutrons, surrounded by electrons in orbit. The nucleus is at the center of the atom and contains positively charged protons and uncharged neutrons. Electrons, which are negatively charged, orbit the nucleus in shells or energy levels.
'''

4. INPUT IN THE FORM OF ENGLISH TEXT <br>
In this case, there is no need for any conversion. The query entered by the user will simply be sent to the LLM and the answer will be returned to the user.